In [ ]:
import time
import json

def run_tests_debug_prompts(tests):
    """
    Executa APENAS a IA 1 (Schema/Plano) e constrói o prompt final 
    que seria enviado para a IA 2 (Cypher), sem executá-la.
    """
    print(f"\n🚀 INICIANDO DEBUG DE PROMPTS ({len(tests)} testes)")
    print("⚠️  Modo: IA 2 (Geração) e Neo4j (Execução) estão DESATIVADOS.")
    
    t_start_suite = time.time()

    for i, item in enumerate(tests, 1):
        q = item["q"]
        tag = item.get("tag", f"test_{i:03d}")
        
        print("\n" + "█" * 80)
        print(f"🔹 TESTE [{i:03d}] TAG: {tag}")
        print(f"❓ PERGUNTA: {q}")
        print("-" * 80)

        try:
            # 1. Executa IA 1 (Schema)
            # Isso é necessário pois o prompt depende das entidades identificadas aqui
            t0 = time.time()
            aischema = get_ai_schema_dict(q) 
            t_ia1 = time.time() - t0

            # 2. Verifica se a pergunta requer query
            needs_query = aischema.get("needs_query", False)
            
            # Mostra o Schema resumido
            print(f"🧠 [IA 1 - SCHEMA] (Tempo: {t_ia1:.2f}s)")
            print(json.dumps(aischema, indent=2, ensure_ascii=False))
            print("-" * 40)

            # 3. Constrói o Prompt (RAG + Template)
            if needs_query:
                # Chama diretamente a função interna de construção do prompt
                # definida na Célula 2
                full_prompt = build_cypher_prompt(q, aischema)
                
                print("📝 [PROMPT GERADO PARA IA 2]:")
                print("vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv")
                print(full_prompt)
                print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
                print(f"ℹ️  Tamanho do Prompt: ~{len(full_prompt)} caracteres")
            else:
                print("⛔ [SEM PROMPT]")
                print("   A IA 1 decidiu que esta é uma pergunta conceitual/conversacional.")
                print("   Nenhuma query Cypher seria gerada.")

        except Exception as e:
            print(f"❌ ERRO NO TESTE: {e}")
            import traceback
            traceback.print_exc()

    total_time = time.time() - t_start_suite
    print("\n" + "=" * 80)
    print(f"🏁 FIM DA SUITE DE DEBUG. Tempo Total: {total_time:.2f}s")

# ==========================================
# EXECUÇÃO
# ==========================================
if __name__ == "__main__":
    # Certifique-se de que a lista ALL_TESTES está definida antes desta célula
    if 'ALL_TESTES' in globals():
        run_tests_debug_prompts(ALL_TESTES)
    else:
        print("❌ A variável 'ALL_TESTES' não foi encontrada. Defina sua lista de testes antes de rodar.")